In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import math
import mlflow
import optuna
import os

from TMDP import TMDP
from algorithms import *
from model_functions import *
from policy_utils import *
from experiment_result_utils import *
from constants import *

from FrozenLake import *
from CurriculumMPI import CurriculumMPI

In [2]:
is_slippery = False
reward_shape = False
dense_reward = False
num_bins = 0
nrows = 30

num_runs = 10
episodes = 3900000
checkpoint_step=500
test_episodes = 1000

shape_range=(-1,0)
goal_reward = 1.
debug = False
param_decay=True

lam = 1
experiment_results = []
exp_taus = []
tests_returns = []
tests_lens = []

In [3]:
if num_bins == 0:
    reward_shape = dense_reward = False
    discount_tau = True
else:
    reward_shape = dense_reward = True
    discount_tau = False

In [4]:
#Frozen Lake Environment
tau = 0.2
nS = nrows**2
nA = 4
gamma = 0.99
xi = np.ones(nS) * 1/nS

In [5]:
run_name = f"CurrMPI_{num_bins}"

mlflow.set_tracking_uri(MLFLOW_URI)
experiment_name = f"FrozenLake_{nrows}x{nrows}_{num_bins}"
experiment_id = get_or_create_experiment(experiment_name)
mlflow.set_experiment(experiment_name)

save_path = f"results/{experiment_name}/run_{run_name}"
label = run_name.split("_")[0]

In [6]:
model_lr = 0.373715
pol_lr = 0.215723
temp = 5
epochs = 1

final_temp=1e-5
batch_size = 10

check_convergence=False
biased=False

In [7]:
def run_experiment(index, seed, run_name, change_map=False):
    sub_run_name = f"{run_name}_{index}"
    
    with mlflow.start_run(nested=True, run_name=sub_run_name):
        # Environment specific configuration   
        map_seed = seed if change_map else constants.SEEDS[0]
        set_policy_seed(seed)
        env = FrozenLakeEnv(is_slippery=is_slippery, seed=seed, 
                        desc=generate_random_map(nrows, seed=map_seed), 
                        reward_shape=reward_shape,
                        num_bins=num_bins,
                        )
        
        # Environment independent configuration
        tmdp = TMDP(env, xi, tau=tau, 
                    gamma=gamma, discount_tau=discount_tau, 
                    seed=seed)

        tmdp.update_tau(tau)
        curr_MPI = CurriculumMPI(tmdp, checkpoint_step=checkpoint_step)
        curr_MPI.train(model_lr, pol_lr, batch_size=batch_size, 
                        lam=lam, temp=temp, final_temp=final_temp,
                        episodes=episodes, check_convergence=check_convergence,
                        param_decay=param_decay, biased=biased,
                        debug=debug, epochs=epochs,)   
        
        avg_return = np.average(curr_MPI.reward_records[-10:])/batch_size
        
        mlflow.log_metric("Avg Return", avg_return)

        test_policies = test_policies_len(tmdp, curr_MPI.thetas, test_episodes, 1e-5)
        test_policies_return = test_policies[0]
        test_pol_len = test_policies[1]

        result_dict = {
            "thetas" : curr_MPI.thetas,
            "taus" : curr_MPI.taus,
            "reward_records" : curr_MPI.reward_records,
            "test_policies_return" : test_policies_return,
            "index" : index,
            "test_pol_len" : test_pol_len,
        }

        tests_returns.append(test_policies_return)
        experiment_results.append(result_dict)
        tests_lens.append(test_pol_len)
        exp_taus.append(curr_MPI.taus)


In [8]:
def run_experiments(change_map=False, num_runs=10):
    with mlflow.start_run(run_name=run_name):
        
        for i in range(num_runs):
            seed = constants.SEEDS[i]
            run_experiment(i, seed, run_name, change_map)
        
        pad_returns = pad_to_same_length(tests_returns)
        pad_lens = pad_to_same_length(tests_lens)
        pad_taus = pad_to_same_length(exp_taus)

        experiment_dict = {
            "tests_returns": pad_returns,
            "taus": pad_taus,
            "tests_lens": pad_lens,
            "num_runs": num_runs,
            "change_map": change_map,
            "num_bins": num_bins,
            "label": label,
            "checkpoint_step": checkpoint_step,
        }

        mlflow.set_tags(tags={
            "change_map": change_map,
            "seed": seed,
            "tau": tau,
            "gamma": gamma,
            "checkpoint_step": checkpoint_step,
            "test_episodes": test_episodes,
            "dense_reward": dense_reward,
            "shape_range": shape_range,
            "goal_reward": goal_reward,
            "reward_shape": reward_shape,
            "episodes": episodes,
            "model_lr": model_lr,
            "pol_lr": pol_lr,
            "temp": temp,
            "final_temp": final_temp,
            "batch_size": batch_size,
            "epochs": epochs,
            "lam": lam,
        })
        try:
            save_to_mlflow(experiment_dict, run_name)
        except Exception as e:
            print(e)
            print("Something went wrong saving the experiment results to MLFlow.")
            print("Saving locally instead.")
            time.sleep(5)
            save(save_path, experiment_dict)

        rewards_fig = plot_avg_test_return(tests_returns, f"CMPI Avg Return on {num_runs} runs")
        try:
            mlflow.log_figure(figure=rewards_fig, artifact_file="reward_image.png")
        except Exception as e:
            print(e)
            print("Something went wrong saving the figure to MLFlow.")
            print("Saving locally instead.")
            time.sleep(5)
            rewards_fig.savefig(save_path+"/reward_image.png")

In [9]:
run_experiments(change_map=False, num_runs=num_runs)

Current seed for result reproducibility: 2999
Episode: 5000 reward: 0.0 tau 0.2
Episode: 10000 reward: 0.0 tau 0.2
Episode: 15000 reward: 0.0 tau 0.2
Episode: 20000 reward: 0.0 tau 0.2
Episode: 25000 reward: 0.0 tau 0.2
Episode: 30000 reward: 0.0 tau 0.2
Episode: 35000 reward: 0.0 tau 0.2
Episode: 40000 reward: 0.0 tau 0.2
Episode: 45000 reward: 0.0 tau 0.2
Episode: 50000 reward: 0.8 tau 0.2
Episode: 55000 reward: 0.0 tau 0.2
Episode: 60000 reward: 0.0 tau 0.2
Episode: 65000 reward: 0.0 tau 0.2
Episode: 70000 reward: 0.0 tau 0.2
Episode: 75000 reward: 0.0 tau 0.2
Episode: 80000 reward: 0.0 tau 0.2
Episode: 85000 reward: 0.0 tau 0.2
Episode: 90000 reward: 0.0 tau 0.2
Episode: 95000 reward: 0.0 tau 0.2
Episode: 100000 reward: 0.0 tau 0.2
Episode: 105000 reward: 0.0 tau 0.2
Episode: 110000 reward: 0.0 tau 0.2
Episode: 115000 reward: 0.0 tau 0.2
Episode: 120000 reward: 0.0 tau 0.2
Episode: 125000 reward: 0.0 tau 0.2
Episode: 130000 reward: 0.0 tau 0.2
Episode: 135000 reward: 0.0 tau 0.2
Ep